In [ ]:
context = f"""
아래는 사용자의 두 가지 입력 정보입니다.

1️⃣ 텍스트 입력:
{user_text}

2️⃣ 이미지 기반 RAG 결과:
{rag_result}

이 두 자료를 모두 고려해, 사실적이고 상세한 답변을 생성해주세요.
"""

In [1]:
%pip install langchain-community langchain-pinecone langchain-openai pinecone -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 kB 13.6 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os

os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [7]:
import os
from typing import List, Dict
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

# 1. Pinecone 인스턴스 & 인덱스
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV     = "us-east-1"
INDEX_NAME       = "food-index"
EMBED_MODEL      = "text-embedding-3-small"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)

# 벡터스토어 준비
vector_store = PineconeVectorStore(
    index=index,
    embedding=embeddings
)

# 2. 메뉴 이름을 받아 Pinecone에서 검색
def search_menu(menu_name: str, k: int = 1) -> List[Dict]:
    """메뉴명을 임베딩해 Pinecone에서 유사 레시피 k개 반환"""
    return vector_store.similarity_search_with_score(
        query=menu_name,
        k=k
    )

# 3. 검색 결과를 컨텍스트 문자열로 변환
def build_context(matches: List[Dict]) -> str:
    """
    Pinecone 검색 결과(matches)의 metadata에서
    RCP_NM(메뉴명)과 INFO_ENG(칼로리)를 추출해 컨텍스트 문자열 반환
    """
    lines = []
    for doc, score in matches:
        meta = doc.metadata or {}
        name = meta.get("RCP_NM", "알 수 없는 메뉴")
        kcal = meta.get("INFO_ENG", "칼로리 정보 없음")
        lines.append(f"- 메뉴명: {name}, 칼로리: {kcal}")
    return "\n".join(lines)

# 4. 전체 파이프라인 함수
def get_menu_context(menu_name: str) -> str:
    """
    BLIP가 예측한 메뉴명을 입력받아
    Pinecone 검색 → 컨텍스트 문자열 반환
    """
    matches = search_menu(menu_name, k=1)
    return build_context(matches)

# 5. test
if __name__ == "__main__":
    blip_predicted_name = "칼국수"  # BLIP 결과 예시
    context = get_menu_context(blip_predicted_name)
    print("=== 컨텍스트 ===")
    print(context)

=== 컨텍스트 ===
- 메뉴명: 들깨칼국수, 칼로리: 255.6
